In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load DialoGPT-small model and tokenizer
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Check if GPU is available and move model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set chat parameters
max_length = 1000
chat_history_ids = None

def chat_with_bot(user_input):
    global chat_history_ids
    
    # Encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt').to(device)
    
    # Create the attention mask
    new_user_attention_mask = torch.ones(new_user_input_ids.shape, dtype=torch.long, device=device)

    # Append the new user input tokens to the chat history
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
        attention_mask = torch.cat([torch.ones(chat_history_ids.shape, dtype=torch.long, device=device), new_user_attention_mask], dim=-1)
    else:
        bot_input_ids = new_user_input_ids
        attention_mask = new_user_attention_mask

    # Generate a response while limiting the total chat history to 1000 tokens
    chat_history_ids = model.generate(
        bot_input_ids, 
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature=0.8,
        attention_mask=attention_mask
    )
    
    # Extract the AI's response
    ai_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # Optionally, trim conversation history if it gets too long
    if chat_history_ids.shape[1] > 1000:
        chat_history_ids = chat_history_ids[:, -1000:]
    
    return ai_response



In [50]:
from tkinter import *
from tkinter import ttk

def chat(*args):
    # Get the input from the dialog_entry
    user_input = dialog.get()
    
    # Update the chat_history StringVar
    chat_history.set(user_input)
    
    # Clear the entry field after sending the message
    dialog.set("")  # Clear the entry box

# Create the main application window
app = Tk()
app.title("AAI520 Dialog Chat")

mainframe = ttk.Frame(app, padding="50 3 20 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))

# Configure the grid to have 5 columns and 5 rows
for i in range(5):
    mainframe.columnconfigure(i, weight=1)  # Allow columns to expand
    mainframe.rowconfigure(i, weight=1)     # Allow rows to expand

# Row 1: Instructions
label = ttk.Label(mainframe, text="Chat with the AI. Type 'exit', 'quit', or 'end' to finish the conversation.")
label.grid(column=2, row=1, sticky=(W, E))

# Row 2: Chat History
label = ttk.Label(mainframe, text="Chat History:")
label.grid(column=1, row=2, sticky=(W, E))
chat_history = StringVar()
ttk.Label(mainframe, textvariable=chat_history).grid(column=2, row=2, sticky=(W, E))

# Row 3: Input Box
label = ttk.Label(mainframe, text="Input: ")
label.grid(column=1, row=3, sticky=(W, E))

dialog = StringVar()
dialog_entry = ttk.Entry(mainframe, width=150, textvariable=dialog)
dialog_entry.grid(column=2, row=3, sticky=(W, E))

# Chat button
ttk.Button(mainframe, text="Chat", command=chat).grid(column=2, row=4, sticky=(W, E))

# Add padding to all child widgets
for child in mainframe.winfo_children(): 
    child.grid_configure(padx=5, pady=5)

# Set focus to the entry field and bind the Enter key to the chat function
dialog_entry.focus()
app.bind("<Return>", chat)

# Run the application
app.mainloop()
